# Clean youtube/youtube music activity

In [1]:
import pandas as pd
import pytz
import glob
import os

data_file = "/home/haaksk/Downloads/takeout-20260105T143128Z-3-001/Takeout/My Activity/YouTube/MyActivity.json"

In [2]:
# Read the JSON file - Google Takeout activity is in JSON array format
df = pd.read_json(data_file)
df.columns

Index(['header', 'title', 'titleUrl', 'subtitles', 'time', 'products',
       'activityControls', 'details'],
      dtype='object')

In [3]:
df = df[df['title'].str.startswith("Watched")]

In [4]:
oslo_tz = pytz.timezone('Europe/Oslo')

df['time_local'] = pd.to_datetime(df['time'], format='ISO8601', utc=True).dt.tz_convert(oslo_tz)

df['year'] = df['time_local'].dt.year
df['month'] = df['time_local'].dt.month
df['day'] = df['time_local'].dt.day
df['hour'] = df['time_local'].dt.hour


df.drop(columns=['activityControls', 'products', 'details'], errors='ignore', inplace=True)
df.head(2)

,header,title,titleUrl,subtitles,time,time_local,year,month,day,hour
0,YouTube Music,Watched Sunset,https://music.youtube.com/watch?v=kuE-5p7nxxk,"[{'name': 'The Midnight - Topic', 'url': 'http...",2026-01-05T14:19:28.863Z,2026-01-05 15:19:28.863000+01:00,2026,1,5,15
1,YouTube Music,Watched Above the Sky,https://music.youtube.com/watch?v=c_C_1S5s3yQ,"[{'name': 'Majestica - Topic', 'url': 'https:/...",2026-01-05T14:13:37.581Z,2026-01-05 15:13:37.581000+01:00,2026,1,5,15


In [5]:
df_ytm = df[df['header'] == 'YouTube Music'].copy()
df_yt = df[df['header'] != 'YouTube Music'].copy()

In [6]:
df_ytm.shape

(28561, 10)

In [7]:
df_ytm['song'] = df_ytm['title'].str.replace('^Watched ', '', regex=True)
df_ytm = df_ytm.drop('title', axis=1)

def extract_artist(subtitles):
    if pd.isna(subtitles) or not subtitles:
        return None
    # subtitles is a list of dicts, get the first one's 'name'
    artist_full = subtitles[0].get('name', None) if len(subtitles) > 0 else None
    if artist_full and ' - ' in artist_full:
        return artist_full.rsplit(' - ', 1)[0]
    return artist_full

df_ytm['artist'] = df_ytm['subtitles'].apply(extract_artist)
df_ytm.drop(columns=['titleUrl', 'subtitles', 'header', 'time'], errors='ignore', inplace=True)

df_ytm.head()

,time_local,year,month,day,hour,song,artist
0,2026-01-05 15:19:28.863000+01:00,2026,1,5,15,Sunset,The Midnight
1,2026-01-05 15:13:37.581000+01:00,2026,1,5,15,Above the Sky,Majestica
2,2026-01-05 15:09:21.283000+01:00,2026,1,5,15,Ordinary Story,In Flames
3,2026-01-05 15:05:17.536000+01:00,2026,1,5,15,All I Need (with Mahalia & Ty Dolla $ign),Jacob Collier
4,2026-01-05 15:01:52.982000+01:00,2026,1,5,15,Alive,Empire of The Sun


In [8]:
df_ytm.sample(20).to_parquet("../testdata/youtube_music_testdata.parquet")

# Spotify data

In [9]:
spotify_data_folder = "~/Downloads/MyData/"

In [10]:
glob.glob(spotify_data_folder + "endsong_*.json")

[]

In [11]:
# Load all endsong_*.json files
spotify_files = glob.glob(os.path.expanduser(spotify_data_folder) + "endsong_*.json")
dfs = []

for file_path in sorted(spotify_files):
    df_temp = pd.read_json(file_path)
    dfs.append(df_temp)

# Merge all dataframes
df_s = pd.concat(dfs, ignore_index=True)

In [12]:
df_s['time_local'] = pd.to_datetime(df_s['ts'], format='ISO8601', utc=True).dt.tz_convert(oslo_tz)

df_s['year'] = df_s['time_local'].dt.year
df_s['month'] = df_s['time_local'].dt.month
df_s['day'] = df_s['time_local'].dt.day
df_s['hour'] = df_s['time_local'].dt.hour
df_s.rename(columns={
    'master_metadata_track_name': 'song',
    'master_metadata_album_artist_name': 'artist'
}, inplace=True)

df_s = df_s[['time_local', 'year', 'month', 'day', 'hour', 'song', 'artist']]

In [13]:
df_s

,time_local,year,month,day,hour,song,artist
0,2010-04-29 22:10:28+02:00,2010,4,29,22,Countdown [Designer Drugs Remix],Jupiter One
1,2010-04-29 22:17:10+02:00,2010,4,29,22,These Woods Breathe Evil,Swallow The Sun
2,2010-04-29 22:20:46+02:00,2010,4,29,22,Falling World,Swallow The Sun
3,2010-04-29 22:20:47+02:00,2010,4,29,22,...and Heavens Cried Blood,Swallow The Sun
4,2010-04-29 22:20:48+02:00,2010,4,29,22,Sleepless Swans,Swallow The Sun
...,...,...,...,...,...,...,...
281068,2018-03-13 13:25:43+01:00,2018,3,13,13,III Ways to Epica,Kamelot
281069,2018-03-13 13:28:23+01:00,2018,3,13,13,Sea Of Machines,Ayreon
281070,2018-03-13 13:30:55+01:00,2018,3,13,13,One Night Stand,Vidar Villa
281071,2018-03-13 13:31:57+01:00,2018,3,13,13,Herregud,Vidar Villa


# Final merge of spotify and youtube music data

In [14]:
print(df_s.shape)
print(df_ytm.shape)

(281073, 7)
(28561, 7)


In [31]:
df_tot = pd.concat([df_s, df_ytm])

In [32]:
df_tot.groupby('song').size().sort_values(ascending=False).head(10)

song
Cut Your Teeth - Kygo Remix    626
Firestone                      568
Hiertebrand                    490
Running in the Night           477
Alive                          472
Klatremusvise                  471
Benighted (Remastered)         469
Epsilon - Original Mix         451
På Låven Sitter Nissen         449
Miami 82 - Kygo Remix          430
dtype: int64

In [33]:
df_tot.groupby('artist').size().sort_values(ascending=False).head(10)

artist
Thorbjørn Egner      7833
Opeth                4590
DragonForce          4587
Kaptein Sabeltann    4364
Power Quest          4257
John Mayer           4188
Kygo                 4180
Empire of the Sun    4137
Montée               4079
Steel Panther        3916
dtype: int64

In [36]:
df_tot[df_tot['artist'] == 'Ghost'].groupby('year').size().sort_index()

year
2013      2
2014    866
2015    814
2016    429
2017     55
2018      2
2019     13
2020     27
2021     10
2022      3
2023    511
2024    333
2025    504
2026      9
dtype: int64

In [37]:
df_tot.to_parquet("~/Downloads/tot.parquet")